# Parte 3 - Modelo 1

## Importo librerias y Data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
train_transaction = pd.read_csv("/content/gdrive/MyDrive/org_datos/TP3 - MACHINE LEARNING/train_transaction.csv")
train_identity = pd.read_csv("/content/gdrive/MyDrive/org_datos/TP3 - MACHINE LEARNING/train_identity.csv")

In [ ]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [ ]:
df_train = train_transaction.merge(train_identity, left_on="TransactionID", right_on="TransactionID", how="outer")
train_transaction = None
train_identity = None

## Separo Train y Validation

In [ ]:
quantile_70 = df_train.TransactionDT.quantile(.7)

In [ ]:
X_train= df_train.loc[df_train['TransactionDT'] < quantile_70]

In [ ]:
Y_train = X_train["isFraud"]

In [ ]:
X_train['TransactionDT'] = X_train['TransactionDT']/60

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X_validation = df_train.loc[df_train['TransactionDT'] > quantile_70]

In [ ]:
Y_validation = X_validation["isFraud"]

In [ ]:
X_validation['TransactionDT'] = X_validation['TransactionDT']/60

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
quantile_70 = None
df_train = None

In [ ]:
X_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,1440.000000,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,1440.016667,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,1441.150000,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,1441.650000,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,1441.766667,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [ ]:
features_importantes = ["ProductCD","TransactionAmt","TransactionID","isFraud","TransactionDT",'V233','V232','V231','V177','V219','V217','V218','V147','V179','DeviceInfo','V239','id_03','V273','V167','V185',
 'V275','V237','V236','V142','id_09',"DeviceType","id_31","id_33",'P_emaildomain','R_emaildomain',"card1","card2","card3","card4","card5","card6","addr1",
 "addr2","dist1","dist2",'M1','M2','M3','M4','M5','M6','M7','M8','M9',"id_30","id_35",	"id_36",	"id_37",	"id_38","id_29","id_12",
 "D1","D2","D3","D4","D5","D6","D7","D8","D9","D10","D11","D12","D13","D14","D15"]

In [ ]:
low_card = ['ProductCD','card4','card6','M1','M2','M3','M4','M5','M6','M7','M8','M9','id_12','id_15','id_16','id_23','id_27','id_28','id_29','id_34','id_35','id_36','id_37','id_38'
,'DeviceType']

In [ ]:
high_card = ['P_emaildomain','R_emaildomain','id_30','id_33','DeviceInfo']

In [ ]:
X_trainV2 = X_train[features_importantes]

In [ ]:
X_trainV2.select_dtypes(include = ["object"])

,ProductCD,DeviceInfo,DeviceType,id_31,id_33,P_emaildomain,R_emaildomain,card4,card6,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_30,id_35,id_36,id_37,id_38,id_29,id_12
0,W,NaN,NaN,NaN,NaN,NaN,NaN,discover,credit,T,T,T,M2,F,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,W,NaN,NaN,NaN,NaN,gmail.com,NaN,mastercard,credit,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,W,NaN,NaN,NaN,NaN,outlook.com,NaN,visa,debit,T,T,T,M0,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,W,NaN,NaN,NaN,NaN,yahoo.com,NaN,mastercard,debit,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,SAMSUNG SM-G892A Build/NRD90M,mobile,samsung browser 6.2,2220x1080,gmail.com,NaN,mastercard,credit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Android 7.0,T,F,T,T,NotFound,NotFound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413373,W,NaN,NaN,NaN,NaN,hotmail.com,NaN,visa,debit,T,F,F,NaN,NaN,F,F,F,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413374,W,NaN,NaN,NaN,NaN,hotmail.com,NaN,visa,debit,T,T,T,M0,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413375,W,NaN,NaN,NaN,NaN,gmail.com,NaN,visa,debit,T,T,T,M0,T,T,F,F,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413376,W,NaN,NaN,NaN,NaN,gmail.com,NaN,mastercard,debit,T,T,T,M0,T,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_trainV2[["id_12"]].value_counts()

id_12   
NotFound    95365
Found       14546
dtype: int64

In [ ]:
low_card2 = ["ProductCD","DeviceType","card4","card6",'M1','M2','M3','M4','M5','M6','M7','M8','M9',"id_35","id_36","id_37","id_38","id_12","id_29"]

In [ ]:
high_card2 = ["id_31","DeviceInfo",'P_emaildomain','R_emaildomain',"P_emaildomain","id_30"]

In [ ]:
len(low_card2+high_card2)

25

In [ ]:
X_trainV2.DeviceInfo.replace(np.nan,"unknown",inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
vectorizer = CountVectorizer(max_features=10)
X = vectorizer.fit_transform(X_trainV2.DeviceInfo)

In [ ]:
len(features_importantes)

71

## Train

In [ ]:
dum_train = pd.get_dummies(X_train[low_card])

In [ ]:
X_train = pd.concat([X_train,dum_train],axis=1)
dum_train = None

In [ ]:
X_train.id_31.replace(np.nan,"unknown",inplace=True)

In [ ]:
vectorizer = CountVectorizer(max_features=25)
X = vectorizer.fit_transform(X_train.id_31)

In [ ]:
X_train[vectorizer.get_feature_names_out()] = X.toarray()

In [ ]:
for i in high_card:
  Mean_encoded_subject = X_train.groupby([i])['isFraud'].mean().to_dict()
  X_train[i] =  X_train[i].map(Mean_encoded_subject)

In [ ]:
X_train.drop(["isFraud","id_31"], axis=1, inplace=True)
X_train.drop(low_card, axis=1, inplace=True)
X_train.drop(['10', '11', '15', '16', '49', '57', '58', '61', '62', '63', '64','65'], axis=1, inplace=True)

In [ ]:
X_train.replace(np.nan,0,inplace=True)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective ='binary:logistic')

In [ ]:
param_dist = {'max_depth': [3, 4],'learning_rate': [0.1,0.2]}

In [ ]:
clf = RandomizedSearchCV(clf_xgb,param_distributions = param_dist, n_iter=1, cv=2, random_state=1,scoring='roc_auc',n_jobs=1)

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
X_train.head(3)

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,...,id_15_New,id_15_Unknown,id_16_Found,id_16_NotFound,id_23_IP_PROXY:ANONYMOUS,id_23_IP_PROXY:HIDDEN,id_23_IP_PROXY:TRANSPARENT,id_27_Found,id_27_NotFound,id_28_Found,id_28_New,id_29_Found,id_29_NotFound,id_34_match_status:-1,id_34_match_status:0,id_34_match_status:1,id_34_match_status:2,id_35_F,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T,id_38_F,id_38_T,DeviceType_desktop,DeviceType_mobile,android,browser,chrome,desktop,edge,firefox,for,generic,ie,mobile,safari,samsung,unknown
0,2987000,1440.000000,68.5,13926,0.0,150.0,142.0,315.0,87.0,19.0,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2987001,1440.016667,29.0,2755,404.0,150.0,102.0,325.0,87.0,0.0,0.0,0.044041,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2987002,1441.150000,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,0.0,0.089014,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## Validation

In [ ]:
dum_val = pd.get_dummies(X_validation[low_card])

In [ ]:
X_validation = pd.concat([X_validation,dum_val],axis=1)
dum_val = None

In [ ]:
X_validation.id_31.replace(np.nan,"unknown",inplace=True)

In [ ]:
vectorizer = CountVectorizer(max_features=25)
X = vectorizer.fit_transform(X_validation.id_31)

In [ ]:
X_validation[vectorizer.get_feature_names_out()] = X.toarray()

In [ ]:
for i in high_card:
  X_validation[i] =  X_validation[i].map(Mean_encoded_subject)

In [ ]:
X_validation[["card6_charge card","id_34_match_status:-1"]] = 0

In [ ]:
X_validation.drop(["52","59","60","66","ios","isFraud","id_31"], axis=1, inplace=True)
X_validation.drop(low_card, axis=1, inplace=True)
X_validation.drop(['10', '11', '16', '49', '63', '64','65'], axis=1, inplace=True)

In [ ]:
X_validation = X_validation[list(X_train.columns.values)]

In [ ]:
X_validation.replace(np.nan,0,inplace=True)

In [ ]:
# con Random
roc_auc_score(Y_validation,clf.predict_proba(X_validation)[:,1])

0.882516717650185

In [ ]:
X_validation.head(3)

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,...,id_15_New,id_15_Unknown,id_16_Found,id_16_NotFound,id_23_IP_PROXY:ANONYMOUS,id_23_IP_PROXY:HIDDEN,id_23_IP_PROXY:TRANSPARENT,id_27_Found,id_27_NotFound,id_28_Found,id_28_New,id_29_Found,id_29_NotFound,id_34_match_status:-1,id_34_match_status:0,id_34_match_status:1,id_34_match_status:2,id_35_F,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T,id_38_F,id_38_T,DeviceType_desktop,DeviceType_mobile,android,browser,chrome,desktop,edge,firefox,for,generic,ie,mobile,safari,samsung,unknown
413378,3400378,173966.716667,49.0,7239,452.0,150.0,117.0,264.0,87.0,5.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,53.0,53.0,0.0,0.0,0.0,0.0,56.0,56.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
413379,3400379,173966.950000,1265.5,18227,583.0,150.0,226.0,472.0,87.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,15.0,14.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
413380,3400380,173967.383333,50.0,11919,170.0,150.0,224.0,126.0,87.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0


# Saco Columnas

In [ ]:
X_train.head()

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,...,id_29_NotFound,id_34_match_status:-1,id_34_match_status:0,id_34_match_status:1,id_34_match_status:2,id_35_F,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T,id_38_F,id_38_T,DeviceType_desktop,DeviceType_mobile,10,11,15,16,49,57,58,61,62,63,64,65,android,browser,chrome,desktop,edge,firefox,for,generic,ie,mobile,safari,samsung,unknown
0,2987000,1440.000000,68.5,13926,0.0,150.0,142.0,315.0,87.0,19.0,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2987001,1440.016667,29.0,2755,404.0,150.0,102.0,325.0,87.0,0.0,0.0,0.044041,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2987002,1441.150000,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,0.0,0.089014,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2987003,1441.650000,50.0,18132,567.0,150.0,117.0,476.0,87.0,0.0,0.0,0.023649,0.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,84.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2987004,1441.766667,50.0,4497,514.0,150.0,102.0,420.0,87.0,0.0,0.0,0.044041,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
X_validation.P_emaildomain.unique()

array([0.])